<a href="https://colab.research.google.com/github/norojordan/big-data-challenge/blob/main/amazon_books.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.0.3'
# spark_version = 'spark-3.2.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:10 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages [787 k

In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-05-24 00:30:44--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  --.-KB/s    in 0.09s   

2022-05-24 00:30:44 (10.2 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [4]:
from pyspark import SparkFiles
# Load in Amazon book reviews from S3 into a DataFrame
url = "https://s3.amazonaws.com/nj-hw-bucket/amazon_reviews_us_Books_v1_02.tsv.gz"
spark.sparkContext.addFile(url)

books_df = spark.read.option('header', 'true').csv(SparkFiles.get("amazon_reviews_us_Books_v1_02.tsv.gz"), inferSchema=True, sep='\t',timestampFormat="yyyy-mm-dd")
books_df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   12076615| RQ58W7SMO911M|0385730586|     122662979|Sisterhood of the...|           Books|          4|            2|          3|   N|                N|this book was a g...|this boook was a ...|2005-01-14 00:10:00|
|         US|   12703090|  RF6IUKMGL8SF|0811828964|      56191234|The Bad Girl's Gu...| 

In [5]:
# Count the number of reviews 
print(books_df.count())

3105520


## Drop duplicates and incomplete rows

In [6]:
books_df = books_df.dropna()
books_df = books_df.dropDuplicates()
print(books_df.count())

3105360


## Examine the schema

In [7]:
books_df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: timestamp (nullable = true)



## Create a new DataFrame for review info

In [8]:
review_id_info = books_df.select(["review_id","customer_id", "product_id", "product_parent", "review_date"])
review_id_info.show(5)

+--------------+-----------+----------+--------------+-------------------+
|     review_id|customer_id|product_id|product_parent|        review_date|
+--------------+-----------+----------+--------------+-------------------+
|R100OKUUTFVT5N|   52114047|0345465199|     135464941|2003-01-08 00:11:00|
|R101J7BKYSSR02|   53069720|0451409272|     218977436|2000-01-10 00:07:00|
|R102CVI8UNG8SL|   52528158|0312230788|     784321176|2002-01-19 00:01:00|
|R102ZPIP863QFN|   52868158|0374527814|     695466664|2002-01-29 00:08:00|
|R1035MZOAUGOVN|   52712162|0671002392|     199249779|2003-01-14 00:12:00|
+--------------+-----------+----------+--------------+-------------------+
only showing top 5 rows



## Create a new DataFrame for products info

In [10]:
products_info=books_df.select(["product_id", "product_title"])
products_info.show(5)

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|0345465199|            Full Cry|
|0451409272|Jimmy the Wags: S...|
|0312230788|Elizabeth Bishop'...|
|0374527814|    Almost No Memory|
|0671002392|The Heart of the ...|
+----------+--------------------+
only showing top 5 rows



## Create a new DataFrame for customers info

In [11]:
# Total customers count 
customer_count = books_df.select(["customer_id"]).count()
customer_count

3105360

In [12]:
customer_books = books_df.groupBy('customer_id').count()
customer_books.show(5)

+-----------+-----+
|customer_id|count|
+-----------+-----+
|   34134831|    7|
|   51768240|    6|
|   52755143|    2|
|   53046320|   27|
|   51024587|    3|
+-----------+-----+
only showing top 5 rows



In [13]:
# Change column name to match schema
customer_books = customer_books.withColumnRenamed("count", "customer_count") 
customer_books.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   34134831|             7|
|   51768240|             6|
|   52755143|             2|
|   53046320|            27|
|   51024587|             3|
|   32305670|             1|
|   52992442|             1|
|   13554691|             1|
|   31317401|             2|
|   38269359|             1|
|   52375293|            28|
|   53028330|             1|
|   51303293|            11|
|   52424779|             5|
|   40981459|             1|
|   49148093|            10|
|   45912147|             1|
|   52591432|             2|
|   40135561|             2|
|   51210576|             4|
+-----------+--------------+
only showing top 20 rows



## Create a new DataFrame for vine info

In [14]:
vine_info = books_df.select(["review_id", "star_rating", "helpful_votes", "total_votes","vine"])
vine_info.show(5)

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R100OKUUTFVT5N|          4|            9|         10|   N|
|R101J7BKYSSR02|          5|            4|          4|   N|
|R102CVI8UNG8SL|          5|            1|          1|   N|
|R102ZPIP863QFN|          5|            5|          5|   N|
|R1035MZOAUGOVN|          3|            0|          1|   N|
+--------------+-----------+-------------+-----------+----+
only showing top 5 rows



## Write DataFrame to RDS

In [ ]:
# Configuration for RDS instance
mode="append"
jdbc_url = "jdbc:postgresql://mypostgreshwdb.cf9s0qdbvngm.us-east-1.rds.amazonaws.com/my_data_HW_db"
config = {"user":"xxx",
          "password": "",
          "driver":"org.postgresql.Driver"}

In [ ]:
# Write DataFrame to pgAdmin table

review_id_info.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [ ]:
# Write DataFrame to table

products_info.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [ ]:
# Write DataFrame to table

customer_books.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [ ]:
# Write DataFrame to table

vine_info.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)